In [1]:
import os
import pandas as pd
from qub_canvas_helper.assignments import CanvasAssignmentManager

# Canvas Login Details

In [6]:
# Where on the internet is canvas
canvas_domain = "https://qub.instructure.com"

# Access token is now in .env under CANVAS_API_TOKEN
access_token = os.getenv("CANVAS_API_TOKEN")

# Check this is set up properly
if not access_token:
    raise ValueError("Missing Canvas API token!")

# Configure Details

Specify the course. Some of the code will not work in a familiarisation environment as the has no sis_student_ids.

The course_id can be found in the url of the module: https://canvas.qub.ac.uk/courses/27862

In [2]:
course_id = 27862

Load the Excel file containing the list of students, which assignments they will have and when.

FORMAT: id (student ID) | name | "Date of assignment"

In [4]:
df_students = pd.read_excel("custom_assignments_test.xlsx")
#df_students['id'] = df_students['id'].astype('string') # stops an error later on
df_students.head()

,id,name,2024-10-24 00:00:00,2024-11-01 00:00:00
0,1491,Matt Damon,P1,P2
1,1490,Jodie Foster,P1,P2
2,1492,Tom Hanks,P1,P2
3,1493,Liam Neeson,P2,P1
4,45,Notareal Person,P2,P1


# Start Talking to Canvas

Instantiate the class using the data provided above

In [7]:
assignment_manager = CanvasAssignmentManager(canvas_domain, access_token, course_id)

Return a dataframe of the students enrolled on canvas.

This is filtered for active students only.

Be patient, this can take a while for large modules.

In [8]:
assignment_manager.get_students_in_module()

,id,name,created_at,sortable_name,short_name,sis_user_id,integration_id
0,1491,Matt Damon,2018-05-09T10:06:55+01:00,"Damon, Matt",Matt Damon,FA_Student3,None
1,1490,Jodie Foster,2018-05-09T10:06:32+01:00,"Foster, Jodie",Jodie Foster,FA_Student2,None
2,1492,Tom Hanks,2018-05-09T10:07:15+01:00,"Hanks, Tom",Tom Hanks,FA_Student4,None
3,1493,Liam Neeson,2018-05-09T10:07:35+01:00,"Neeson, Liam",Liam Neeson,FA_Student5,None
4,1489,Kate Winslet,2018-05-09T10:06:07+01:00,"Winslet, Kate",Kate Winslet,FA_Student1,None


Return a dataframe of all the assignments in this module 

In [9]:
assignment_manager.get_assignments_in_module(published_status = "all") # 'all', 'published', or 'unpublished'

,id,name,due_at,points_possible,published
0,181313,Example Assignment,2024-10-31T23:59:59Z,100.0,True
1,203603,Assignment 1,2024-11-01T23:59:59Z,0.0,True
2,203604,Assignment 2,2024-11-01T23:59:59Z,0.0,True


In [10]:
assignment_dict = {"P1":"203603", 
                   "P2":"203604"} 

Check that there are no missing or extra students on Canvas or in the Excel sheet. 

Works by comparing sis_student_id (Canvas) to the id specified in Excle. This must be the first column in the Excel sheet.

Be patient, this can take a while for large modules. It will not work for the familiarisation area.

In [11]:
assignment_manager.check_student_enrollment(df_students)

Students missing in Canvas:
- Matt Damon (ID: 1491)
- Jodie Foster (ID: 1490)
- Tom Hanks (ID: 1492)
- Liam Neeson (ID: 1493)
- Notareal Person (ID: 45)
Students missing in Excel:
- Damon, Matt (ID: FA_Student3)
- Foster, Jodie (ID: FA_Student2)
- Hanks, Tom (ID: FA_Student4)
- Neeson, Liam (ID: FA_Student5)
- Winslet, Kate (ID: FA_Student1)


# Start Making Changes on Canvas

Assign students to assignments based on the Excel sheet provided.

A check will first be made that there are no missing or extra students using the function above, this can be ignored if required.

In [12]:
assignment_manager.assign_students_to_assignments(df_students, assignment_dict, check_enrollment=False)

ValueError: invalid literal for int() with base 10: 'FA_Student3'

If any mistakes have been made, then individual assignment to students can be removed, leaving the assignment as it was to begin with.

This can done for a single assignment or multiple assignments if entered as a list

In [13]:
assignment_manager.remove_student_assignments([203603,203604])

Completed removing overrides for assignments: [203603, 203604]


### Check that all students have been assigned

If the excel list was incorrect, some students may not have been assigned. We can take a look.

In [1]:
assignment_manager.check_student_assignment_completions(assignment_dict)

NameError: name 'assignment_manager' is not defined